<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/saskeli/x/blob/master/pandas3.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

|                                       -                                       |                                       -                                       |                                       -                                       |
|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------|
| [Exercise 1 (split date continues)](<#Exercise-1-(split-date-continues&#41;>) |      [Exercise 2 (cycling weather)](<#Exercise-2-(cycling-weather&#41;>)      |            [Exercise 3 (top bands)](<#Exercise-3-(top-bands&#41;>)            |
|     [Exercise 4 (cyclists per day)](<#Exercise-4-(cyclists-per-day&#41;>)     |  [Exercise 5 (best record company)](<#Exercise-5-(best-record-company&#41;>)  |    [Exercise 6 (suicide fractions)](<#Exercise-6-(suicide-fractions&#41;>)    |
|      [Exercise 7 (suicide weather)](<#Exercise-7-(suicide-weather&#41;>)      |   [Exercise 8 (bicycle timeseries)](<#Exercise-8-(bicycle-timeseries&#41;>)   |              [Exercise 9 (commute)](<#Exercise-9-(commute&#41;>)              |



# Pandas (continues)

In [ ]:
import pandas as pd
import numpy as np

## Catenating datasets

We already saw in the NumPy section how we can catenate arrays along an axis: `axis=0` catenates vertically and `axis=1` catenates horizontally, and so on. With the DataFrames of Pandas it works similarly except that the row indices and the column names require extra attention. Also note a slight difference in the name: `np.concatenate` but `pd.concat`.

Let's start by considering catenation along the axis 0, that is, vertical catenation. We will first make a helper function to easily create DataFrames for testing.

In [ ]:
def makedf(cols, ind):
    data = {c : [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

Next we will create some example DataFrames:

In [ ]:
a=makedf("AB", [0,1])
a

In [ ]:
b=makedf("AB", [2,3])
b

In [ ]:
c=makedf("CD", [0,1])
c

In [ ]:
d=makedf("BC", [2,3])
d

In the following simple case, the `concat` function works exactly as we expect it would:

In [ ]:
pd.concat([a,b])   # The default axis is 0

The next, however, will create duplicate indices:

In [ ]:
r=pd.concat([a,a])
r

In [ ]:
r.loc[0,"A"]

This is not usually what we want! There are three solutions to this. Firstly, deny creation of duplicated indices by giving the `verify_integrity` parameter to the `concat` function:

In [ ]:
try:
    pd.concat([a,a], verify_integrity=True)
except ValueError as e:
    import sys
    print(e, file=sys.stderr)

Secondly, we can ask for automatic renumbering of rows:

In [ ]:
pd.concat([a,a], ignore_index=True)

Thirdly, we can ask for *hierarchical indexing*. The indices can contain multiple levels, but on this course we don't consider hierarchical indices in detail. Hierarchical indices can make a two dimensional array to work like higher dimensional array.

In [ ]:
r2=pd.concat([a,a], keys=['first', 'second'])
r2

In [ ]:
r2["A"]["first"][0]

Everything works similarly, when we want to catenate horizontally:

In [ ]:
pd.concat([a,c], axis=1)

We have so far assumed that when concatenating vertically the columns of both DataFrames are the same, and when joining horizontally the indices are the same. This is, however, not required:

In [ ]:
pd.concat([a,d], sort=False)    # sort option is used to silence a deprecation message

It expanded the non-existing cases with `NaN`s. This method is called an *outer join*, which forms the union of columns in the two DataFrames. The alternative is *inner join*, which forms the intersection of columns:

In [ ]:
pd.concat([a,d], join="inner")

#### <div class="alert alert-info">Exercise 1 (split date continues)</div>

Write function `split_date_continues` that does

* read the bicycle data set
* clean the data set of columns/rows that contain only missing values
* drops the `Päivämäärä` column and replaces it with its splitted components as before

Use the `concat` function to do this. The function should return a DataFrame with 25 columns (first five related to the date and then the rest 20 conserning the measument location.

<hr/>

## Merging dataframes

Merging combines two DataFrames based on some common field.

Let's recall the earlier DataFrame about wages and ages of persons:

In [ ]:
df = pd.DataFrame([[1000, "Jack", 21], [1500, "John", 29]], columns=["Wage", "Name", "Age"])
df

Now, create a new DataFrame with the occupations of persons:

In [ ]:
df2 = pd.DataFrame({"Name" : ["John", "Jack"], "Occupation": ["Plumber", "Carpenter"]})
df2

The following function call will merge the two DataFrames on their common field, and, importantly, will keep the indices *aligned*. What this means is that even though the names are listed in different order in the two frames, the merge will still give correct result.

In [ ]:
pd.merge(df, df2)

This was an example of a simple one-to-one merge, where the keys in the `Name` columns had 1-to-1 correspondence. Sometimes not all the keys appear in both DataFrames:

In [ ]:
df3 = pd.concat([df2, pd.DataFrame({ "Name" : ["James"], "Occupation":["Painter"]})], ignore_index=True)
df3

In [ ]:
pd.merge(df, df3)                # By default an inner join is computed

In [ ]:
pd.merge(df, df3, how="outer")   # Outer join

Also, many-to-one and many-to-many relationships can occur in merges:

In [ ]:
books = pd.DataFrame({"Title" : ["War and Peace", "Good Omens", "Good Omens"] , 
                      "Author" : ["Tolstoi", "Terry Pratchett", "Neil Gaiman"]})
books

In [ ]:
collections = pd.DataFrame([["Oodi", "War and Peace"],
                           ["Oodi", "Good Omens"],
                           ["Pasila", "Good Omens"],
                           ["Kallio", "War and Peace"]], columns=["Library", "Title"])
collections

All combinations with matching keys (`Title`) are created:

In [ ]:
libraries_with_books_by = pd.merge(books, collections)
libraries_with_books_by

#### <div class="alert alert-info">Exercise 2 (cycling weather)</div>

Merge the processed cycling data set (from the previous exercise) and weather data set along the columns year, month, and day. Note that the names of these columns might be different in the two tables: use the `left_on` and `right_on` parameters. Then drop useless columns 'm', 'd', 'Time', and 'Time zone'.

Write function `cycling_weather` that reads the data sets and returns the resulting DataFrame.

<hr/>

#### <div class="alert alert-info">Exercise 3 (top bands)</div>

Merge the DataFrames UK top40 and the bands DataFrame that are stored in the `src` folder.
Do all this in the parameterless function `top_bands`, which should return the merged DataFrame.
Use the `left_on` and `right_on` parameters to `merge`. Test your function from the `main` function.

<hr/>

## Aggregates and groupings

Let us use again the weather dataset. First, we make the column names a bit more uniform and concise. For example the columns `Year`, `m`, and `d` are not uniformly named.

We can easily change the column names with the `rename` method of the DataFrame. Note that we cannot directly change the index `wh.columns` as it is immutable.

In [ ]:
wh = pd.read_csv("https://www.cs.helsinki.fi/u/jttoivon/dap/data/fmi/kumpula-weather-2017.csv")

In [ ]:
wh3 = wh.rename(columns={"m": "Month", "d": "Day", "Precipitation amount (mm)" : "Precipitation", 
                         "Snow depth (cm)" : "Snow", "Air temperature (degC)" : "Temperature"})
wh3.head()

Pandas has an operation that splits a DataFrame into groups, performs some operation on each of the groups, and then combines the result from each group into a resulting DataFrame. This split-apply-combine functionality is really flexible and powerful operation. In Pandas you start by calling the `groupby` method, which splits the DataFrame into groups. In the following example the rows that contain measurements from the same month belong to the same group:

In [ ]:
groups = wh3.groupby("Month")
groups

Nothing happened yet, but the `groupby` object knows how the division into groups is done. This is called a lazy operation. We can query the number of groups in the `groupby` object:

In [ ]:
len(groups)

We can iterate through all the groups:

In [ ]:
for key, group in groups:
    print(key, len(group))

In [ ]:
groups.get_group(2)                 # Group with index two is February

The `groupby` object functions a bit like a DataFrame, so some operations which are allowed for DataFrames are also allowed for the `groupby` object. For example, we can get a subset of columns:

In [ ]:
groups["Temperature"]

For each DataFrame corresponding to a group the Temperature column was chosen. Still nothing was shown, because we haven't applied any operation on the groups.

The common methods also include the aggregation methods. Let's try to apply the `mean` aggregation:

In [ ]:
groups["Temperature"].mean()

Now what happened was that after the mean aggregation was performed on each group, the results were automatically combined into a resulting DataFrame. Let's try some other aggregation:

In [ ]:
groups["Precipitation"].sum()

Ok, the -1.0 values in the Precipitation field are causing trouble here, let's convert them to zeros:

In [ ]:
wh4 = wh3.copy()
wh4.loc[wh4.Precipitation == -1, "Precipitation"] = 0
wh4.loc[wh4.Snow == -1, "Snow"] = 0
wh4.head()

In [ ]:
wh4.groupby("Month")["Precipitation"].sum()

### Other ways to operate on groups

The aggregations are not the only possible operations on groups. The other possibilities are filtering, transformation, and application.

In **filtering** some of the groups can be filtered out.

In [ ]:
def myfilter(df):                                     # The filter function must return a boolean value
    return df["Precipitation"].sum() >= 150

wh4.groupby("Month").filter(myfilter)                 # Filter out months with total precipitation less that 150 mm

In a **transformation** each group's DataFrame is manipulated in a way that retains its shape. An example of centering values, so that the deviations from the monthly means are shown:

In [ ]:
pd.concat([wh4.iloc[:, 0:3], 
           wh4.groupby("Month")[["Precipitation", "Snow", "Temperature"]].transform(lambda x : x - x.mean())], 
          axis=1)

The **apply** method is very generic and only requires that for each group's DataFrame the given function returns a DataFrame, Series, or a scalar. In the following example, we sort within each group by the temperature:

In [ ]:
wh4.groupby("Month").apply(lambda df : df.sort_values("Temperature"))

#### <div class="alert alert-info">Exercise 4 (cyclists per day)</div>

This exercise can give two points at maximum!

Part 1.

Read, clean and parse the bicycle data set as before. Group the rows by year, month, and day. Get the sum for each group.
Make function `cyclists_per_day` that does the above. The function should return a DataFrame.
Make sure that the columns Hour and Weekday are not included in the returned DataFrame.

Part 2.

In the `main` function, using the function `cyclists_per_day`, get the daily counts.  The index of the DataFrame now consists of tuples (Year, Month, Day). Then restrict this data to August of year 2017, and plot this data. Don't forget to call the `plt.show` function of matplotlib. The x-axis should have ticks from 1 to 31, and there should be a curve to each measuring station. Can you spot the weekends?

<hr/>

#### <div class="alert alert-info">Exercise 5 (best record company)</div>

We use again the UK top 40 data set from the first week of 1964 in the `src` folder. Here we define "goodness" of a record company (`Publisher`) based on the sum of the weeks on chart (WoC) of its singles. Return a DataFrame of the singles by the best record company (a subset of rows of the original DataFrame). Do this with function `best_record_company`.

<hr/>

#### <div class="alert alert-info">Exercise 6 (suicide fractions)</div>

Load the suicide data set from `src` folder. This data was originally downloaded from [Kaggle](https://www.kaggle.com/szamil/who-suicide-statistics). Kaggle contains lots of interesting open data sets.

Write function `suicide_fractions` that loads the data set and returns a Series that has the country as the (row) index and as the column the mean fraction of suicides per population in that country. In other words, the value is the average of suicide fractions. The information about year, sex and age is not used.

<hr/>

#### <div class="alert alert-info">Exercise 7 (suicide weather)</div>

Copy the function `suicide fractions` from the previous exercise. 

Implement function `suicide_weather` as described below.
We use the dataset of average temperature (over years 1961-1990) in different countries from `src/List_of_countries_by_average_yearly_temperature.html` (https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature) .
You can use the function `pd.read_html` to get all the tables from a html page. By default `pd.read_html` does not know which row contains column headers and which column contains row headers. Therefore, you have to give both `index_col` and `header` parameters to `read_html`. Maku sure you use the country as the (row) index for both of the DataFrames. What is the [Spearman correlation](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) between these variables? Use the `corr` method of Series object. Note the the two Series need not be sorted as the indices of the rows (country names) are used to align them.

The return value of the function `suicide_weather` is a tuple (suicide_rows, temperature_rows, common_rows, spearman_correlation)
The output from the `main` function should be of the following form:

```
Suicide DataFrame has x rows
Temperature DataFrame has x rows
Common DataFrame has x rows
Spearman correlation: x.x
```

You might have trouble when trying to convert the temperatures to float. The is because the negative numbers on that html page use a special *unicode minus sign*, which looks typographically nice, but the float constructor cannot interpret it as a minus sign. You can try out the following example:

In [ ]:
s="\u2212" "5"   # unicode minus sign and five
print(s)
try:
    float(s)
except ValueError as e:
    import sys
    print(e, file=sys.stderr)
        

But if we explicitly convert unicode minus sign to normal minus sign, it works:

In [ ]:
float(s.replace("\u2212", "-"))

<hr/>

## Time series

If a measurement is made at certain points in time, the resulting values with their measurement times is called a time series. In Pandas a Series whose index consists of dates/times is a time series.

Let's make a copy of the DataFrame that we can mess with:

In [ ]:
wh2 = wh3.copy()
wh2.columns

The column names `Year`, `Month`, and `Day` are now in appropriate form for the `to_datetime` function. It can convert these fields into a timestamp series, which we will add to the DataFrame.

In [ ]:
wh2["Date"] = pd.to_datetime(wh2[["Year", "Month", "Day"]])
wh2.head()

We can now drop the useless fields:

In [ ]:
wh2=wh2.drop(columns=["Year", "Month", "Day"])
wh2.head()

The following method call will set the Date field as the index of the DataFrame.

In [ ]:
wh2 = wh2.set_index("Date")
wh2.head()

We can now easily get a set of rows using date slices:

In [ ]:
wh2["2017-01-15":"2017-02-03"]

By using the `date_range` function even more complicated sets can be formed. The following gets all the Mondays of July:

In [ ]:
r=pd.date_range("2017-07-01", "2017-07-31", freq="w-mon")
r

In [ ]:
wh2.index.difference(r)

In [ ]:
wh2.loc[r,:]

The following finds all the business days (Monday to Friday) of July:

In [ ]:
pd.date_range("2017-07-01", "2017-07-31", freq="b")

We can get a general idea about the `Temperature` column by plotting it. Note how the index time series is shown nicely on the x-axis.

In [ ]:
%matplotlib inline
wh2["Temperature"].plot();

The graph looks a bit messy at this level of detail. We can smooth it by taking averages over a sliding window of length 30 days:

In [ ]:
rolling = wh2.Temperature.rolling(30, center=True)
rolling

In [ ]:
data = pd.DataFrame({"Temperature" : wh2.Temperature, "Rolling mean" : rolling.mean()})
data.plot();

#### <div class="alert alert-info">Exercise 8 (bicycle timeseries)</div>

Write function `bicycle_timeseries` that

* reads the data set
* cleans it
* turns its `Päivämäärä` column into (row) DatetimeIndex (that is, to row names) of that DataFrame
* returns the new DataFrame

<hr/>

#### <div class="alert alert-info">Exercise 9 (commute)</div>

In function `commute` do the following:

Use the function `bicycle_timeseries` to get the bicycle data. Restrict to August 2017, group by the weekday, aggregate by summing. Set the `Weekday` column to numbers from one to seven. Then set the column `Weekday` as the (row) index. Return the resulting DataFrame from the function.

In the `main` function plot the DataFrame. Xticklabels should be the weekdays. Don't forget to call `show` function!

If you want the xticklabels to be `['Mon', 'Tue', 'Wed', 'Thu', 'Fr', 'Sat', 'Sun']` instead of numbers (1,..,7), then it may get a bit messy. There seems to be a problem with non-numeric `x` values. You could try the following after plotting, but you don't have to:

```python
weekdays="x mon tue wed thu fri sat sun".title().split()
plt.gca().set_xticklabels(weekdays)
```

<hr/>

## Additional information

[Pandas cheat sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) Summary of most important Pandas' functions and methods.

Read the article [Tidy Data](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf). The article uses the statistical software R as an example, but the ideas are relevant in general. Pandas operations maintain data in the tidy format.

Pandas handles only one dimensional data (Series) and two dimensional data (DataFrame). While you can use [hierarchical indices](http://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#hierarchical-indexing-multiindex) to simulate higher dimensional arrays, you should use the [xarray](http://xarray.pydata.org/en/stable/index.html) library, if you need proper higher-dimensional arrays with labels. It is basically a cross between NumPy and Pandas.



<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/saskeli/x/blob/master/pandas3.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
